In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.mediation import Mediation
import pandas as pd
import numpy as np
from sklearn.decomposition import FactorAnalysis

# 0. Preparing dataset

In [2]:
# Reading and filtering datasets
data = pd.read_csv("casuality_data_final_factor_analyzer.csv")
heart_df = data.filter(regex='heart')
cardio_cmr_df = data.filter(regex='cardio_cmr')
X1 = pd.concat([heart_df, cardio_cmr_df], axis=1)
X2 = data.filter(regex='brain')
g_VRF = data["g_VRF"]
X1.shape, X2.shape, g_VRF.shape

((2065, 639), (2065, 744), (2065,))

In [3]:
# Extracting latent factor for heart and brain
factor_heart = FactorAnalysis(n_components=1)
factor_heart = factor_heart.fit_transform(X1, g_VRF)
factor_brain = FactorAnalysis(n_components=1)
factor_brain = factor_brain.fit_transform(X2, g_VRF)
factor_heart = factor_heart[:, 0]
factor_brain = factor_brain[:, 0]
factor_heart.shape, factor_brain.shape, g_VRF.shape

((2065,), (2065,), (2065,))

In [4]:
data['factor_heart'] = factor_heart
data['factor_brain'] = factor_brain

In [5]:
data_new = data.filter(['factor_heart', 'factor_brain', 'g_VRF'], axis=1)
data_new.head()

,factor_heart,factor_brain,g_VRF
0,-1.364912,1.062501,-0.270503
1,-1.186440,-1.240734,-0.399763
2,1.164212,-1.648003,1.179964
3,0.793579,-0.003127,-1.170220
4,-0.661120,0.919291,-0.291274


# 1. Running regressions (VRF->Brain with Heart mediation)

In [6]:
# inputs
X = data["g_VRF"]
M = factor_heart
Y = factor_brain

A mediation analysis is comprised of three sets of regression: X → Y, X → M, and X + M → Y

## 1.1 X -> Y

We want X to affect Y. If there is no relationship between X and Y, there is nothing to mediate.

In [7]:
X = sm.add_constant(X)
est = sm.OLS(Y, X)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     87.87
Date:                Tue, 25 Aug 2020   Prob (F-statistic):           1.77e-20
Time:                        01:27:02   Log-Likelihood:                -2883.8
No. Observations:                2065   AIC:                             5772.
Df Residuals:                    2063   BIC:                             5783.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.561e-16      0.022   7.25e-15      1.0

We can see how the coefficient for g_VRF (-0.1895) is significant. Therefore, X affects Y.

## 1.2 X -> M

We want X to affect M. If X and M have no relationship, M is just a third variable that may or may not be associated with Y. A mediation makes sense only if X affects M.

In [8]:
X = sm.add_constant(X)
est = sm.OLS(M, X)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     155.6
Date:                Tue, 25 Aug 2020   Prob (F-statistic):           1.72e-34
Time:                        01:27:02   Log-Likelihood:                -2853.8
No. Observations:                2065   AIC:                             5712.
Df Residuals:                    2063   BIC:                             5723.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.922e-17      0.021   2.32e-15      1.0

We can see how the coefficient for g_VRF (-0.2485) is significant. Therefore, X affects M.

## 1.3 X + M -> Y

We want M to affect Y, but X to no longer affect Y (or X to still affect Y but in a smaller magnitude). If a mediation effect exists, the effect of X on Y will disappear (or at least weaken) when M is included in the regression. The effect of X on Y goes through M.

If the effect of X on Y completely disappears, M fully mediates between X and Y (full mediation). If the effect of X on Y still exists, but in a smaller magnitude, M partially mediates between X and Y (partial mediation).

In [9]:
mod = smf.ols(formula='factor_brain ~ g_VRF + factor_heart', data=data_new)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:           factor_brain   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     47.08
Date:                Tue, 25 Aug 2020   Prob (F-statistic):           1.02e-20
Time:                        01:27:02   Log-Likelihood:                -2880.8
No. Observations:                2065   AIC:                             5768.
Df Residuals:                    2062   BIC:                             5784.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     1.561e-16      0.022   7.26e-15   

We can see how the coefficient for g_VRF (-0.2031) is still significant. Therefore, X still affects Y. However, the mediation effect of factor_heart is very low (-0.0550). Therefore, the heart does not play an important mediating role between VRFs and the brain.

# 2. Mediation Analysis

Once we find these relationships, we want to see if this mediation effect is statistically significant (different from zero or not). To do so, there are two main approaches: the Sobel test (Sobel, 1982) and bootstrapping (Preacher & Hayes, 2004). In R, you can use sobel() in ‘multilevel’ package for the Sobel test and mediate() in ‘mediation’ package for bootstrapping. Because bootstrapping is strongly recommended in recent years (although Sobel test was widely used before), I’ll show only the bootstrapping method in this example.


mediate() takes two model objects as input (X → M and X + M → Y) and we need to specify which variable is an IV (treatment) and a mediator (mediator). For bootstrapping, set boot = TRUE and sims to at least 500. After running it, look for ACME (Average Causal Mediation Effects) in the results and see if it’s different from zero.

In [10]:
from statsmodels.stats.mediation import Mediation
outcome_model = sm.OLS.from_formula("factor_brain ~  g_VRF + factor_heart", data = data_new)
mediator_model = sm.OLS.from_formula("factor_heart ~ g_VRF", data = data_new)
med = Mediation(outcome_model, mediator_model, "g_VRF", mediator = "factor_heart")
med_result = med.fit(n_rep = 500, method = 'bootstrap')
print(np.round(med_result.summary(), decimals = 3))

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)               0.014           0.003           0.026    0.012
ACME (treated)               0.014           0.003           0.026    0.012
ADE (control)               -0.202          -0.246          -0.160    0.000
ADE (treated)               -0.202          -0.246          -0.160    0.000
Total effect                -0.189          -0.232          -0.148    0.000
Prop. mediated (control)    -0.070          -0.149          -0.017    0.012
Prop. mediated (treated)    -0.070          -0.149          -0.017    0.012
ACME (average)               0.014           0.003           0.026    0.012
ADE (average)               -0.202          -0.246          -0.160    0.000
Prop. mediated (average)    -0.070          -0.149          -0.017    0.012


Note that the Total Effect in the summary (-0.189) is the coefficient in the first step: a total effect of X on Y (without M). The direct effect (ADE, -0.202) is the coefficient in the third step: a direct effect of X on Y after taking into account a mediation (indirect) effect of M. Finally, the mediation effect (ACME) is the total effect minus the direct effect. The goal of mediation analysis is to obtain this indirect effect and see if it’s statistically significant.

# 3. References

[1]: [Kim Bommae (2016)](https://data.library.virginia.edu/introduction-to-mediation-analysis/). Introduction to Mediation Analysis